`Classification based on features`

In [27]:
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut, LeavePGroupsOut
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from glmnet.logistic import LogitNet
from sklearn.model_selection import GroupKFold

import matplotlib.pyplot as plt

In [2]:
res_feat = np.load('./Features/res-feat.npy')
#vgg_feat = np.load('./Features/vgg-feat.npy')

In [4]:
## features
size = 200
index = res_feat[:,-2] != 0

res_feat_ = res_feat[index, :2048]
#vgg_feat_ = vgg_feat[index, :512]
label_ = res_feat[index, 2048].astype(int)
group_ = res_feat[index, 2049].astype(int)

In [5]:
## 7-fold cross validations
kf = GroupKFold(n_splits=7)
kf.get_n_splits(group_)
## Leave one out cross-validation
logo = LeaveOneGroupOut()
logo.get_n_splits(groups=group_)

21

`KNN classifier`

`LOO`

In [40]:
best_model = {}
best_model['pic_acc'] = 0
best_model['sample_acc'] = 0
best_model['k'] = 0
list_pic_acc = []

for k in [1, 2,3, 4, 8, 16, 32, 64]:
    
    temp_results = np.zeros((21,2))
    tt = 0
    
    for train_index, test_index in logo.split(X = res_feat_, y = label_, groups = group_):

        X_train, X_test = res_feat_[train_index], res_feat_[test_index]
        y_train, y_test = label_[train_index], label_[test_index]

        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X = X_train, y = y_train)
        y_pred = model.predict(X = X_test)
        temp_results[tt, 0] = (y_pred == y_test).mean() # sample_acc

        pic_pred = y_pred.mean() > 0
        pic_test = y_test.mean() > 0 
        temp_results[tt, 1] = (pic_pred == pic_test) # pic_acc
        
        tt += 1
        
    y_acc, pic_acc = temp_results.mean(axis=0)
    list_pic_acc.append(pic_acc)
    if pic_acc > best_model['pic_acc']:
        best_model['pic_acc'] = pic_acc
        best_model['sample_acc'] = y_acc
        best_model['k'] = k
        best_model['model'] = model
    

In [41]:
best_model

{'k': 1,
 'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=1, p=2,
            weights='uniform'),
 'pic_acc': 0.7142857142857143,
 'sample_acc': 0.65142857142857136}

`3-Folds`

In [30]:
best_model = {}
best_model['pic_acc'] = 0
best_model['sample_acc'] = 0
best_model['k'] = 0
list_pic_acc = []

for k in [1, 2, 3, 4, 8, 16, 32, 64]:
    
    temp_results = np.zeros((7,2))
    tt = 0
    
    for train_index, test_index in kf.split(X = res_feat_, y = label_, groups = group_):

        X_train, X_test = res_feat_[train_index], res_feat_[test_index]
        y_train, y_test = label_[train_index], label_[test_index]

        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X = X_train, y = y_train)
        y_pred = model.predict(X = X_test)
        temp_results[tt, 0] = (y_pred == y_test).mean() # sample_acc

        pic_pred = np.array([y_pred[size*i: size*(i+1)].mean() > 0 for i in range(3)])
        pic_test = np.array([y_test[size*i: size*(i+1)].mean() > 0 for i in range(3)])
        temp_results[tt, 1] = (pic_pred == pic_test).mean() # pic_acc
        
        tt += 1
        
    y_acc, pic_acc = temp_results.mean(axis=0)
    list_pic_acc.append(pic_acc)
    if pic_acc > best_model['pic_acc']:
        best_model['pic_acc'] = pic_acc
        best_model['sample_acc'] = y_acc
        best_model['k'] = k
        best_model['model'] = model
    

In [31]:
best_model

{'k': 1,
 'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=1, p=2,
            weights='uniform'),
 'pic_acc': 0.7142857142857143,
 'sample_acc': 0.63214285714285712}

`Logistic Regression`

`3-Folds`

In [38]:
best_model = {}
best_model['pic_acc'] = 0
best_model['sample_acc'] = 0
best_model['inv_lam'] = 0
list_pic_acc = []

for inv_lam in np.linspace(start=3, stop=10, num=25):
    
    tt = 0
    temp_results = np.zeros((7,2))

    for train_index, test_index in kf.split(X = res_feat_, y = label_, groups = group_):

        X_train, X_test = res_feat_[train_index], res_feat_[test_index]
        y_train, y_test = label_[train_index], label_[test_index]

        model = LogisticRegression(penalty='l1',C= 2 ** inv_lam)
        model.fit(X = X_train, y = y_train)
        y_pred = model.predict(X = X_test)
        temp_results[tt, 0] = (y_pred == y_test).mean() # sample_acc

        pic_pred = np.array([y_pred[size*i: size*(i+1)].mean() > 0 for i in range(3)])
        pic_test = np.array([y_test[size*i: size*(i+1)].mean() > 0 for i in range(3)])
        temp_results[tt, 1] = (pic_pred == pic_test).mean() # pic_acc
        
        tt += 1
        
    y_acc, pic_acc = temp_results.mean(axis=0)
    list_pic_acc.append(pic_acc)
    if pic_acc > best_model['pic_acc']:
        best_model['pic_acc'] = pic_acc
        best_model['sample_acc'] = y_acc
        best_model['inv_lam'] = inv_lam
        best_model['model'] = model

    #    print('Pic: %2d, Pred: %2f, Truth: %2f' % (group_[test_index[0]]+1, logo_results[tt,0], logo_results[tt,1]))

In [39]:
best_model

{'inv_lam': 6.791666666666667,
 'model': LogisticRegression(C=110.78867980878633, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'pic_acc': 0.76190476190476186,
 'sample_acc': 0.69714285714285729}

`LOO`

In [21]:
best_model = {}
best_model['pic_acc'] = 0
best_model['sample_acc'] = 0
best_model['inv_lam'] = 0
list_pic_acc = []

for inv_lam in np.linspace(start=-2, stop=10, num=20):
    
    tt = 0
    temp_results = np.zeros((21,2))

    for train_index, test_index in logo.split(X = res_feat_, y = label_, groups = group_):

        X_train, X_test = res_feat_[train_index], res_feat_[test_index]
        y_train, y_test = label_[train_index], label_[test_index]

        model = LogisticRegression(penalty='l1',C= 2 ** inv_lam)
        model.fit(X = X_train, y = y_train)
        y_pred = model.predict(X = X_test)
        temp_results[tt, 0] = (y_pred == y_test).mean() # sample_acc

        pic_pred = y_pred.mean() > 0
        pic_test = y_test.mean() > 0 
        temp_results[tt, 1] = (pic_pred == pic_test) # pic_acc
        
        tt += 1
        
    y_acc, pic_acc = temp_results.mean(axis=0)
    list_pic_acc.append(pic_acc)
    if pic_acc > best_model['pic_acc']:
        best_model['pic_acc'] = pic_acc
        best_model['sample_acc'] = y_acc
        best_model['inv_lam'] = inv_lam
        best_model['model'] = model

    #    print('Pic: %2d, Pred: %2f, Truth: %2f' % (group_[test_index[0]]+1, logo_results[tt,0], logo_results[tt,1]))

In [23]:
best_model

{'inv_lam': -5.5416666666666661,
 'model': LogisticRegression(C=0.021468025741532649, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'pic_acc': 0.80952380952380953,
 'sample_acc': 0.71357142857142852}

`SVM`

`LOO`

In [55]:
best_model = {}
best_model['pic_acc'] = 0
best_model['sample_acc'] = 0
best_model['lam'] = 0
list_pic_acc = []

for lam in np.linspace(start=-5, stop=10, num=15):
    
    tt = 0
    temp_results = np.zeros((21,2))

    for train_index, test_index in logo.split(X = res_feat_, y = label_, groups = group_):

        X_train, X_test = res_feat_[train_index], res_feat_[test_index]
        y_train, y_test = label_[train_index], label_[test_index]

        model = SVC(C= 2 ** lam)
        model.fit(X = X_train, y = y_train)
        y_pred = model.predict(X = X_test)
        temp_results[tt, 0] = (y_pred == y_test).mean() # sample_acc

        pic_pred = y_pred.mean() > 0
        pic_test = y_test.mean() > 0 
        temp_results[tt, 1] = (pic_pred == pic_test) # pic_acc
        
        tt += 1
        
    y_acc, pic_acc = temp_results.mean(axis=0)
    list_pic_acc.append(pic_acc)
    if pic_acc > best_model['pic_acc']:
        best_model['pic_acc'] = pic_acc
        best_model['sample_acc'] = y_acc
        best_model['lam'] = lam
        best_model['model'] = model

    #    print('Pic: %2d, Pred: %2f, Truth: %2f' % (group_[test_index[0]]+1, logo_results[tt,0], logo_results[tt,1]))

In [56]:
best_model

{'lam': -0.71428571428571441,
 'model': SVC(C=0.60950682710223769, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 'pic_acc': 0.66666666666666663,
 'sample_acc': 0.65809523809523829}

`7-Folds`

In [50]:
best_model = {}
best_model['pic_acc'] = 0
best_model['sample_acc'] = 0
best_model['lam'] = 0
list_pic_acc = []

for lam in np.linspace(start=-5, stop=5, num=20):
    
    tt = 0
    temp_results = np.zeros((7,2))

    for train_index, test_index in kf.split(X = res_feat_, y = label_, groups = group_):

        X_train, X_test = res_feat_[train_index], res_feat_[test_index]
        y_train, y_test = label_[train_index], label_[test_index]

        model = SVC(C= 2 ** lam, kernel='linear')
        model.fit(X = X_train, y = y_train)
        y_pred = model.predict(X = X_test)
        temp_results[tt, 0] = (y_pred == y_test).mean() # sample_acc

        pic_pred = np.array([y_pred[size*i: size*(i+1)].mean() > 0 for i in range(3)])
        pic_test = np.array([y_test[size*i: size*(i+1)].mean() > 0 for i in range(3)])
        temp_results[tt, 1] = (pic_pred == pic_test).mean() # pic_acc
        
        tt += 1
        
    y_acc, pic_acc = temp_results.mean(axis=0)
    list_pic_acc.append(pic_acc)
    if pic_acc > best_model['pic_acc']:
        best_model['pic_acc'] = pic_acc
        best_model['sample_acc'] = y_acc
        best_model['lam'] = lam
        best_model['model'] = model

    #    print('Pic: %2d, Pred: %2f, Truth: %2f' % (group_[test_index[0]]+1, logo_results[tt,0], logo_results[tt,1]))

In [53]:
best_model

{'lam': -5.0,
 'model': SVC(C=0.03125, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 'pic_acc': 0.7142857142857143,
 'sample_acc': 0.65500000000000014}